In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
semana = input("Ingrese Semana:")

Ingrese Semana:43


In [3]:
mainpath = "C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Data"
ganancias_path = "Detalle Ganancia S"+semana+".csv"
deuda_path = "Detalle Deuda S"+semana+".csv"
garantias_path = "Garantias Turnos S"+semana+".xlsx"
conexion_path = "Info Conexion.csv"
ordenes_path = "Detalle Ordenes S"+semana+".csv"

fp_earn = os.path.join(mainpath,ganancias_path)
fp_debt = os.path.join(mainpath,deuda_path)
fp_garantias = os.path.join(mainpath,garantias_path)
fp_conexion = os.path.join(mainpath,conexion_path)
fp_ordenes = os.path.join(mainpath,ordenes_path)

In [4]:
df_ganancia = pd.read_csv(fp_earn, parse_dates = ['created_at'])
df_conexion = pd.read_csv(fp_conexion, parse_dates = ['fecha'])
df_ldam = pd.read_excel(fp_garantias, sheet_name = "La Dehesa AM")
df_ldpm = pd.read_excel(fp_garantias, sheet_name = "La Dehesa PM")
df_autos = pd.read_excel(fp_garantias, sheet_name = "Autos")
df_vinaAm = pd.read_excel(fp_garantias, sheet_name = "Viña AM")
df_vinaPm = pd.read_excel(fp_garantias, sheet_name = "Viña PM")
df_vina19 = pd.read_excel(fp_garantias, sheet_name = "Viña 19-23")
df_vina13 = pd.read_excel(fp_garantias, sheet_name = "Viña 13-1")
df_ordenes = pd.read_csv(fp_ordenes, parse_dates = ['CREATED_AT'])

In [5]:
ldam = df_ldam['ID'].tolist()
ldpm = df_ldpm['ID'].tolist()
#ldesp = df_ldpm['Martes Miercoles'].tolist()
autos = df_autos['ID'].tolist()
vinaAm = df_vinaAm['ID'].tolist()
vinaPm = df_vinaPm['ID'].tolist()
vina19 = df_vina19['ID'].tolist()
vina13 = df_vina13['ID'].tolist()

In [6]:
rts_gar = ldam + ldpm
rts_gar = rts_gar + autos
rts_gar = rts_gar + vinaAm + vinaPm + vina19 + vina13 #+ldesp
ld = ldam + ldpm

In [7]:
df_conexion = df_conexion[df_conexion['skid'].isin(rts_gar)]

In [8]:
def calculoGarantia(tiempo, valor):
    if(tiempo >= 30):
        return valor

In [9]:
def garantiasHorarias(df_conexion, horaA, horaC, dias, lista, valor):
    if(horaA < horaC):
        dft1 = df_conexion[df_conexion['hora']>=horaA]
        dft1 = dft1[dft1['hora']<horaC]
    if(horaA > horaC):
        dft2 = df_conexion[df_conexion['hora']>horaA] 
        dft3 = df_conexion[df_conexion['hora']<horaC] 
        dft1 = pd.merge(dft2,dft3,how = 'outer', on=dft2.columns.tolist())
    dft1.insert(2,'Dia',dft1['fecha'].apply(lambda x: x.strftime("%w")))
    dft1 = dft1[dft1['skid'].isin(lista)]
    dft1 = dft1[dft1['Dia'].astype(int).isin(dias)]
    dft1.insert(8, "Garantia", dft1['time'].apply(lambda x: calculoGarantia(x,valor)))
    dft1 = dft1.fillna(0)
    dft1 = dft1.groupby(['skid','fecha','Dia']).sum().reset_index()
    dft1.insert(8,'Garantia Efectiva',np.maximum(0,dft1.Garantia-dft1.ganancia))
    dft1 = dft1.drop(labels = 'hora', axis = 1)
    return dft1

In [10]:
df_ordenes.head()

,STOREKEEPER_ID,CREATED_AT,NAME,MODEL_ID,ADDRESS,VERTICAL,TOTAL,DESCRIPTION
0,1,2018-10-26 20:08:27.720,service_cost,374443,"Enrique Foster Sur 207, Las Condes",rappifavor,2651.6,NaN
1,1,2018-10-26 23:54:36.964,service_cost,376919,"Enrique Foster Sur 207, Las Condes",rappifavor,2651.6,NaN
2,10042,2018-10-24 18:56:20.461,service_cost,356832,Brown Norte,restaurantes,1400.0,NaN
3,10045,2018-10-23 16:12:54.402,service_cost,351050,Diagonal Paraguay 24,super,2100.0,NaN
4,10045,2018-10-23 16:12:54.402,service_cost,351050,Diagonal Paraguay 24,super,2100.0,NaN


In [11]:
df_ganancia.insert(2,'fecha',df_ganancia['created_at'].apply(lambda x: x.strftime("%Y-%M-%d")))
df_ganancia.insert(3,'hora',df_ganancia['created_at'].apply(lambda x: x.strftime("%H")))
df_ganancia = df_ganancia.rename(columns ={'storekeeper_id':'skid'})

In [12]:
df_ganancia.head()

,skid,created_at,fecha,hora,name,model_id,total,description
0,1,2018-10-26 20:08:27.720,2018-08-26,20,service_cost,374443,2651.6,NaN
1,1,2018-10-26 23:54:36.964,2018-54-26,23,service_cost,376919,2651.6,NaN
2,10042,2018-10-24 18:56:20.461,2018-56-24,18,service_cost,356832,1400.0,NaN
3,10045,2018-10-23 16:12:54.402,2018-12-23,16,service_cost,351050,2100.0,NaN
4,10045,2018-10-23 16:12:54.418,2018-12-23,16,tip,351050,1000.0,NaN


In [13]:
df_ganancia['hora'] = df_ganancia['hora'].apply(lambda x: int(x))

In [14]:
#Garantia la Dehesa 9 a 9, Lunes a Domingo
dias = [0,1,2,3,4,5,6]
dft_ldam = garantiasHorarias(df_conexion,9,21, dias, ldam, 2800)

In [15]:
#Garantia la Dehesa PM, grupo que bajo el miercoles.
dias = [2,3]
#dft_ldesp = garantiasHorarias(df_conexion,12,24, dias, ldesp, 2800)

In [16]:
#Garantia la Dehesa 12 a 12, Lunes a Jueves y Domingo y 15 a 3, Viernes y Sabado
dias = [0,1,2,3,4]
df_ldpm1 = garantiasHorarias(df_conexion, 12, 24, dias, ldpm, 2800)
df_ldpm2 = garantiasHorarias(df_conexion, 15, 3, [5,6], ldpm, 2800)
dft_ldpm = pd.merge(df_ldpm1, df_ldpm2, how='outer', on=df_ldpm1.columns.tolist())

In [17]:
#Garantia 10 a 10 Domingo y 9 a 9 Sabado Autos
df_autos1 = garantiasHorarias(df_conexion, 9, 21, [6], autos, 3334)
df_autos2 = garantiasHorarias(df_conexion, 10, 22, [0], autos, 3334)
df_autos = pd.merge(df_autos1, df_autos2, how = 'outer', on = df_autos1.columns.tolist())

In [18]:
df_ordLD = df_ordenes[df_ordenes['STOREKEEPER_ID'].isin(ld)]
df_ordLD = df_ordLD[df_ordLD['NAME']=='service_cost']
df_ordLD = df_ordLD.groupby(['STOREKEEPER_ID','MODEL_ID', 'NAME', 'ADDRESS','CREATED_AT']).sum()

In [19]:
#Garantias Viña 4 Turnos
dias = [0,1,2,3,4,5,6]
dft_vina1 = garantiasHorarias(df_conexion,8,20, dias, vinaAm, 2800)
dft_vina2 = garantiasHorarias(df_conexion,11,23, dias, vinaPm, 2800)
dft_vina3 = garantiasHorarias(df_conexion,19,23, dias, vina19, 2800)
dft_vina4 = garantiasHorarias(df_conexion,13,1, dias, vina13, 2800)
dft_vina5 = garantiasHorarias(df_conexion,11,23, [4,5], [12830], 2800)

In [20]:
writer = pd.ExcelWriter('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Garantias S'+semana+'.xlsx')
dft_ldam.to_excel(writer,'Garantia La Dehesa AM')
dft_ldpm.to_excel(writer,'Garantia La Dehesa PM')
#dft_ldesp.to_excel(writer,'Garantia La Dehesa Esp')
df_autos.to_excel(writer,'Garantias Autos')
dft_vina1.to_excel(writer,'Garantias Viña AM')
dft_vina2.to_excel(writer,'Garantias Viña PM')
dft_vina3.to_excel(writer,'Garantias Viña 19-23')
dft_vina4.to_excel(writer,'Garantias Viña 13-1')
dft_vina5.to_excel(writer,'Garantias Viña RT 12830')
writer.save()
writer.close()
print("Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Garantias S"+semana+".xlsx")

Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S43/Garantias S43.xlsx


In [21]:
df_ordLD.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Ordenes LD S'+semana+'.xlsx')